# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
models.search_threshold = -1
models.double_dummy_eval = False
# Filter to remove hands not consistent with the bidding. Used during play
models.bid_accept_play_threshold = 0.10
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies
#sampler.use_distance = False
#sampler.bidding_threshold_sampling = 0.25


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-8210000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-8210000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-1278000


### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:

hands = ['AKQJT2.AQT2.A6.6', '.K964.KQ93.KJ532', '96543.5.J74.AT94', '87.J873.T852.Q87']
hands = ['AKQJTx.AQTx.Ax.x', 'xxx.Kx.KT8x.KJxx', 'x.J98xxxx.9.QTxx', '98x..QJxxxx.A98x']

In [4]:
# North is dealer
dealer = 0
bidder_bots = [BotBid([True, True], hand, models, sampler, i, dealer, False) for i, hand in enumerate(hands)]

In [5]:
auction = [] 

turn_i = 0 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    #if step== 5:
    #     bid.bid='X'
    if step== 1:
         bid.bid='PASS'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

[10  8 13]
{'bid': '2C', 'who': 'NN', 'candidates': [{'call': '2C', 'insta_score': 0.794}], 'hcp': [6.2, 8.5, 5.5], 'shape': [2.3, 2.8, 3.7, 4.0, 2.7, 2.8, 3.6, 3.7, 2.4, 2.9, 3.7, 3.9]}
RHO 2C
[ 2  3 13]
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 0.996}], 'hcp': [5.1, 3.5, 21.2], 'shape': [3.2, 3.8, 3.1, 2.9, 3.1, 3.4, 3.3, 3.1, 3.3, 3.9, 2.9, 2.9]}
PAR 2C
RHO PASS
[11 17 12]
{'bid': '2D', 'who': 'NN', 'candidates': [{'call': '2D', 'insta_score': 0.942}], 'hcp': [7.3, 22.2, 6.1], 'shape': [3.7, 2.3, 3.9, 3.1, 4.0, 2.5, 3.5, 3.0, 3.6, 2.7, 3.8, 2.9]}
LHO 2C
PAR PASS
RHO 2D
[ 2  3 14]
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 0.991}], 'hcp': [22.1, 6.3, 5.0], 'shape': [3.6, 3.9, 2.3, 3.2, 3.2, 3.7, 2.7, 3.4, 3.3, 3.5, 2.6, 3.5]}
LHO PASS
PAR 2D
RHO PASS
[13 17 18]
{'bid': '2S', 'who': 'NN', 'candidates': [{'call': '2S', 'insta_score': 0.977}], 'hcp': [7.4, 5.6, 6.8], 'shape': [2.5, 2.9, 3.7, 3.9, 2.6, 2.9, 3.7, 3.8, 2.5,